# CycleGAN model

> Defines the CycleGAN model architecture.

In [1]:
# default_exp models.cyclegan

In [2]:
#export
from fastai.vision.all import *
from fastai.basics import *
from typing import List
from fastai.vision.gan import *
from huggingface_hub import PyTorchModelHubMixin

In [3]:
#hide
from upit.models.junyanz import define_G, define_D

In [4]:
#hide
from nbdev.showdoc import *

We use the models that were introduced in the [cycleGAN paper](https://arxiv.org/abs/1703.10593).

## Generator

In [5]:
#export
def convT_norm_relu(ch_in:int, ch_out:int, norm_layer:nn.Module, ks:int=3, stride:int=2, bias:bool=True):
    return [nn.ConvTranspose2d(ch_in, ch_out, kernel_size=ks, stride=stride, padding=1, output_padding=1, bias=bias),
            norm_layer(ch_out), nn.ReLU(True)]

In [6]:
show_doc(convT_norm_relu,title_level=3)

<h3 id="convT_norm_relu" class="doc_header"><code>convT_norm_relu</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>convT_norm_relu</code>(**`ch_in`**:`int`, **`ch_out`**:`int`, **`norm_layer`**:`Module`, **`ks`**:`int`=*`3`*, **`stride`**:`int`=*`2`*, **`bias`**:`bool`=*`True`*)



In [7]:
#export
def pad_conv_norm_relu(ch_in:int, ch_out:int, pad_mode:str, norm_layer:nn.Module, ks:int=3, bias:bool=True, 
                       pad=1, stride:int=1, activ:bool=True, init=nn.init.kaiming_normal_, init_gain:int=0.02)->List[nn.Module]:
    layers = []
    if pad_mode == 'reflection': layers.append(nn.ReflectionPad2d(pad))
    elif pad_mode == 'border':   layers.append(nn.ReplicationPad2d(pad))
    p = pad if pad_mode == 'zeros' else 0
    conv = nn.Conv2d(ch_in, ch_out, kernel_size=ks, padding=p, stride=stride, bias=bias)
    if init:
        if init == nn.init.normal_:
            init(conv.weight, 0.0, init_gain)
        else:
            init(conv.weight)
        if hasattr(conv, 'bias') and hasattr(conv.bias, 'data'): conv.bias.data.fill_(0.)
    layers += [conv, norm_layer(ch_out)]
    if activ: layers.append(nn.ReLU(inplace=True))
    return layers

In [8]:
show_doc(pad_conv_norm_relu,title_level=3)

<h3 id="pad_conv_norm_relu" class="doc_header"><code>pad_conv_norm_relu</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>pad_conv_norm_relu</code>(**`ch_in`**:`int`, **`ch_out`**:`int`, **`pad_mode`**:`str`, **`norm_layer`**:`Module`, **`ks`**:`int`=*`3`*, **`bias`**:`bool`=*`True`*, **`pad`**=*`1`*, **`stride`**:`int`=*`1`*, **`activ`**:`bool`=*`True`*, **`init`**=*`kaiming_normal_`*, **`init_gain`**:`int`=*`0.02`*)



In [9]:
#export
class ResnetBlock(nn.Module):
    "nn.Module for the ResNet Block"
    def __init__(self, dim:int, pad_mode:str='reflection', norm_layer:nn.Module=None, dropout:float=0., bias:bool=True):
        super().__init__()
        assert pad_mode in ['zeros', 'reflection', 'border'], f'padding {pad_mode} not implemented.'
        norm_layer = ifnone(norm_layer, nn.InstanceNorm2d)
        layers = pad_conv_norm_relu(dim, dim, pad_mode, norm_layer, bias=bias)
        if dropout != 0: layers.append(nn.Dropout(dropout))
        layers += pad_conv_norm_relu(dim, dim, pad_mode, norm_layer, bias=bias, activ=False)
        self.conv_block = nn.Sequential(*layers)

    def forward(self, x): return x + self.conv_block(x)

In [10]:
show_doc(ResnetBlock,title_level=3)

<h3 id="ResnetBlock" class="doc_header"><code>class</code> <code>ResnetBlock</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>ResnetBlock</code>(**`dim`**:`int`, **`pad_mode`**:`str`=*`'reflection'`*, **`norm_layer`**:`Module`=*`None`*, **`dropout`**:`float`=*`0.0`*, **`bias`**:`bool`=*`True`*) :: `Module`

nn.Module for the ResNet Block

In [11]:
#export
def resnet_generator(ch_in:int, ch_out:int, n_ftrs:int=64, norm_layer:nn.Module=None, 
                     dropout:float=0., n_blocks:int=9, pad_mode:str='reflection')->nn.Module:
    norm_layer = ifnone(norm_layer, nn.InstanceNorm2d)
    bias = (norm_layer == nn.InstanceNorm2d)
    layers = pad_conv_norm_relu(ch_in, n_ftrs, 'reflection', norm_layer, pad=3, ks=7, bias=bias)
    for i in range(2):
        layers += pad_conv_norm_relu(n_ftrs, n_ftrs *2, 'zeros', norm_layer, stride=2, bias=bias)
        n_ftrs *= 2
    layers += [ResnetBlock(n_ftrs, pad_mode, norm_layer, dropout, bias) for _ in range(n_blocks)]
    for i in range(2):
        layers += convT_norm_relu(n_ftrs, n_ftrs//2, norm_layer, bias=bias)
        n_ftrs //= 2
    layers += [nn.ReflectionPad2d(3), nn.Conv2d(n_ftrs, ch_out, kernel_size=7, padding=0), nn.Tanh()]
    return nn.Sequential(*layers)

In [12]:
show_doc(resnet_generator,title_level=3)

<h3 id="resnet_generator" class="doc_header"><code>resnet_generator</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>resnet_generator</code>(**`ch_in`**:`int`, **`ch_out`**:`int`, **`n_ftrs`**:`int`=*`64`*, **`norm_layer`**:`Module`=*`None`*, **`dropout`**:`float`=*`0.0`*, **`n_blocks`**:`int`=*`9`*, **`pad_mode`**:`str`=*`'reflection'`*)



### Test generator

Let's test for a few things:
1. The generator can indeed be initialized correctly
2. A random image can be passed into the model successfully with the correct size output
3. The CycleGAN generator is equivalent to the [original implementation](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/models/cycle_gan_model.py)

First let's create a random batch:

In [13]:
img1 = torch.randn(4,3,256,256)

In [14]:
m = resnet_generator(3,3)
with torch.no_grad():
    out1 = m(img1)
out1.shape

torch.Size([4, 3, 256, 256])

In [15]:
m_junyanz = define_G(3,3,64,'resnet_9blocks', norm='instance')
with torch.no_grad():
    out2 = m_junyanz(img1)
out2.shape

initialize network with normal


torch.Size([4, 3, 256, 256])

In [16]:
#export
def compare_networks(a,b):
    "A simple function to compare the printed model representations as a proxy for actually comparing two models"
    assert len(a) == len(b)
    for i in range(len(a)):
        assert (a[i].__repr__()==b[i].__repr__()), f"{a[i]} \n and \n {b[i]} \n not equal (position {i})"
    print("Passed!")
    return True

In [17]:
test_eq(out1.shape,img1.shape)
test_eq(out2.shape,img1.shape)
assert compare_networks(list(m_junyanz.children())[0],m)

Passed!


## Discriminator

In [18]:
#export
def conv_norm_lr(ch_in:int, ch_out:int, norm_layer:nn.Module=None, ks:int=3, bias:bool=True, pad:int=1, stride:int=1, 
                 activ:bool=True, slope:float=0.2, init=nn.init.normal_, init_gain:int=0.02)->List[nn.Module]:
    conv = nn.Conv2d(ch_in, ch_out, kernel_size=ks, padding=pad, stride=stride, bias=bias)
    if init:
        if init == nn.init.normal_:
            init(conv.weight, 0.0, init_gain)
        else:
            init(conv.weight)
        if hasattr(conv, 'bias') and hasattr(conv.bias, 'data'): conv.bias.data.fill_(0.)
    layers = [conv]
    if norm_layer is not None: layers.append(norm_layer(ch_out))
    if activ: layers.append(nn.LeakyReLU(slope, inplace=True))
    return layers

In [19]:
show_doc(conv_norm_lr,title_level=3)

<h3 id="conv_norm_lr" class="doc_header"><code>conv_norm_lr</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>conv_norm_lr</code>(**`ch_in`**:`int`, **`ch_out`**:`int`, **`norm_layer`**:`Module`=*`None`*, **`ks`**:`int`=*`3`*, **`bias`**:`bool`=*`True`*, **`pad`**:`int`=*`1`*, **`stride`**:`int`=*`1`*, **`activ`**:`bool`=*`True`*, **`slope`**:`float`=*`0.2`*, **`init`**=*`normal_`*, **`init_gain`**:`int`=*`0.02`*)



In [20]:
#export
def discriminator(ch_in:int, n_ftrs:int=64, n_layers:int=3, norm_layer:nn.Module=None, sigmoid:bool=False)->nn.Module:
    norm_layer = ifnone(norm_layer, nn.InstanceNorm2d)
    bias = (norm_layer == nn.InstanceNorm2d)
    layers = conv_norm_lr(ch_in, n_ftrs, ks=4, stride=2, pad=1)
    for i in range(n_layers-1):
        new_ftrs = 2*n_ftrs if i <= 3 else n_ftrs
        layers += conv_norm_lr(n_ftrs, new_ftrs, norm_layer, ks=4, stride=2, pad=1, bias=bias)
        n_ftrs = new_ftrs
    new_ftrs = 2*n_ftrs if n_layers <=3 else n_ftrs
    layers += conv_norm_lr(n_ftrs, new_ftrs, norm_layer, ks=4, stride=1, pad=1, bias=bias)
    layers.append(nn.Conv2d(new_ftrs, 1, kernel_size=4, stride=1, padding=1))
    if sigmoid: layers.append(nn.Sigmoid())
    return nn.Sequential(*layers)

In [21]:
show_doc(discriminator,title_level=3)

<h3 id="discriminator" class="doc_header"><code>discriminator</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>discriminator</code>(**`ch_in`**:`int`, **`n_ftrs`**:`int`=*`64`*, **`n_layers`**:`int`=*`3`*, **`norm_layer`**:`Module`=*`None`*, **`sigmoid`**:`bool`=*`False`*)



### Test discriminator
Let's test for similar things:
1. The discriminator can indeed be initialized correctly
2. A random image can be passed into the discriminator successfully with the correct size output
3. The CycleGAN discriminator is equivalent to the [original implementation](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/models/cycle_gan_model.py)

In [22]:
d = discriminator(3)
with torch.no_grad():
    out1 = d(img1)
out1.shape

torch.Size([4, 1, 30, 30])

In [23]:
img1 = torch.randn(4,3,256,256)

In [24]:
d_junyanz = define_D(3,64,'basic',norm='instance')
with torch.no_grad():
    out2 = d_junyanz(img1)
out2.shape

initialize network with normal


torch.Size([4, 1, 30, 30])

In [25]:
test_eq(out1.shape,torch.Size([4, 1, 30, 30]))
test_eq(out2.shape,torch.Size([4, 1, 30, 30]))
assert compare_networks(list(d_junyanz.children())[0],d)

Passed!


## Full model

We group two discriminators and two generators in a single model, then a `Callback` (defined in `02_cyclegan_training.ipynb`) will take care of training them properly. We use the `PyTorchModelHubMixin` to provide support for pushing to and loading from the [HuggingFace Hub](https://huggingface.co/docs/hub/main).

In [26]:
#export
class CycleGAN(nn.Module, PyTorchModelHubMixin):
    """
    CycleGAN model. \n
    When called, takes in input batch of real images from both domains and outputs fake images for the opposite domains (with the generators). 
    Also outputs identity images after passing the images into generators that outputs its domain type (needed for identity loss).

    Attributes: \n
    `G_A` (`nn.Module`): takes real input B and generates fake input A \n
    `G_B` (`nn.Module`): takes real input A and generates fake input B \n
    `D_A` (`nn.Module`): trained to make the difference between real input A and fake input A \n
    `D_B` (`nn.Module`): trained to make the difference between real input B and fake input B \n
    """
    def __init__(self, ch_in:int=3, ch_out:int=3, n_features:int=64, disc_layers:int=3, gen_blocks:int=9, lsgan:bool=True, 
                 drop:float=0., norm_layer:nn.Module=None):
        """
        Constructor for CycleGAN model.
        
        Arguments: \n
        `ch_in` (`int`): Number of input channels (default=3) \n
        `ch_out` (`int`): Number of output channels (default=3) \n
        `n_features` (`int`): Number of input features (default=64) \n
        `disc_layers` (`int`): Number of discriminator layers (default=3) \n
        `gen_blocks` (`int`): Number of residual blocks in the generator (default=9) \n
        `lsgan` (`bool`): LSGAN training objective (output unnormalized float) or not? (default=True) \n
        `drop` (`float`): Level of dropout (default=0) \n
        `norm_layer` (`nn.Module`): Type of normalization layer to use in the models (default=None)
        """
        
        super().__init__()
        #G_A: takes real input B and generates fake input A
        #G_B: takes real input A and generates fake input B
        #D_A: trained to make the difference between real input A and fake input A
        #D_B: trained to make the difference between real input B and fake input B
        self.D_A = discriminator(ch_in, n_features, disc_layers, norm_layer, sigmoid=not lsgan)
        self.D_B = discriminator(ch_in, n_features, disc_layers, norm_layer, sigmoid=not lsgan)
        self.G_A = resnet_generator(ch_in, ch_out, n_features, norm_layer, drop, gen_blocks)
        self.G_B = resnet_generator(ch_in, ch_out, n_features, norm_layer, drop, gen_blocks)
        
    
    def forward(self, input):
        """Forward function for CycleGAN model. The input is a tuple of a batch of real images from both domains A and B."""
        real_A, real_B = input
        fake_A, fake_B = self.G_A(real_B), self.G_B(real_A)
        idt_A, idt_B = self.G_A(real_A), self.G_B(real_B) #Needed for the identity loss during training.
        return [fake_A, fake_B, idt_A, idt_B]

In [27]:
show_doc(CycleGAN,title_level=3)

<h3 id="CycleGAN" class="doc_header"><code>class</code> <code>CycleGAN</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>CycleGAN</code>(**`ch_in`**:`int`=*`3`*, **`ch_out`**:`int`=*`3`*, **`n_features`**:`int`=*`64`*, **`disc_layers`**:`int`=*`3`*, **`gen_blocks`**:`int`=*`9`*, **`lsgan`**:`bool`=*`True`*, **`drop`**:`float`=*`0.0`*, **`norm_layer`**:`Module`=*`None`*) :: `Module`

CycleGAN model. 

When called, takes in input batch of real images from both domains and outputs fake images for the opposite domains (with the generators). 
Also outputs identity images after passing the images into generators that outputs its domain type (needed for identity loss).

Attributes: 

`G_A` (`nn.Module`): takes real input B and generates fake input A 

`G_B` (`nn.Module`): takes real input A and generates fake input B 

`D_A` (`nn.Module`): trained to make the difference between real input A and fake input A 

`D_B` (`nn.Module`): trained to make the difference between real input B and fake input B 

In [28]:
show_doc(CycleGAN.__init__)

<h4 id="CycleGAN.__init__" class="doc_header"><code>CycleGAN.__init__</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>CycleGAN.__init__</code>(**`ch_in`**:`int`=*`3`*, **`ch_out`**:`int`=*`3`*, **`n_features`**:`int`=*`64`*, **`disc_layers`**:`int`=*`3`*, **`gen_blocks`**:`int`=*`9`*, **`lsgan`**:`bool`=*`True`*, **`drop`**:`float`=*`0.0`*, **`norm_layer`**:`Module`=*`None`*)

Constructor for CycleGAN model.

Arguments: 

`ch_in` (`int`): Number of input channels (default=3) 

`ch_out` (`int`): Number of output channels (default=3) 

`n_features` (`int`): Number of input features (default=64) 

`disc_layers` (`int`): Number of discriminator layers (default=3) 

`gen_blocks` (`int`): Number of residual blocks in the generator (default=9) 

`lsgan` (`bool`): LSGAN training objective (output unnormalized float) or not? (default=True) 

`drop` (`float`): Level of dropout (default=0) 

`norm_layer` (`nn.Module`): Type of normalization layer to use in the models (default=None)

In [29]:
show_doc(CycleGAN.forward)

<h4 id="CycleGAN.forward" class="doc_header"><code>CycleGAN.forward</code><a href="__main__.py#L41" class="source_link" style="float:right">[source]</a></h4>

> <code>CycleGAN.forward</code>(**`input`**)

Forward function for CycleGAN model. The input is a tuple of a batch of real images from both domains A and B.

In [30]:
show_doc(CycleGAN.push_to_hub)

<h4 id="ModelHubMixin.push_to_hub" class="doc_header"><code>ModelHubMixin.push_to_hub</code><a href="huggingface_hub/hub_mixin.py#L211" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHubMixin.push_to_hub</code>(**`repo_path_or_name`**:`Optional`\[`str`\]=*`None`*, **`repo_url`**:`Optional`\[`str`\]=*`None`*, **`commit_message`**:`Optional`\[`str`\]=*`'Add model'`*, **`organization`**:`Optional`\[`str`\]=*`None`*, **`private`**:`Optional`\[`bool`\]=*`None`*, **`api_endpoint`**:`Optional`\[`str`\]=*`None`*, **`use_auth_token`**:`Union`\[`bool`, `str`, `NoneType`\]=*`None`*, **`git_user`**:`Optional`\[`str`\]=*`None`*, **`git_email`**:`Optional`\[`str`\]=*`None`*, **`config`**:`Optional`\[`dict`\]=*`None`*)

Upload model checkpoint or tokenizer files to the Hub while
synchronizing a local clone of the repo in `repo_path_or_name`.

Parameters:
    repo_path_or_name (`str`, *optional*):
        Can either be a repository name for your model or tokenizer in
        the Hub or a path to a local folder (in which case the
        repository will have the name of that local folder). If not
        specified, will default to the name given by `repo_url` and a
        local directory with that name will be created.
    repo_url (`str`, *optional*):
        Specify this in case you want to push to an existing repository
        in the hub. If unspecified, a new repository will be created in
        your namespace (unless you specify an `organization`) with
        `repo_name`.
    commit_message (`str`, *optional*):
        Message to commit while pushing. Will default to `"add config"`,
        `"add tokenizer"` or `"add model"` depending on the type of the
        class.
    organization (`str`, *optional*):
        Organization in which you want to push your model or tokenizer
        (you must be a member of this organization).
    private (`bool`, *optional*):
        Whether the repository created should be private.
    api_endpoint (`str`, *optional*):
        The API endpoint to use when pushing the model to the hub.
    use_auth_token (`bool` or `str`, *optional*):
        The token to use as HTTP bearer authorization for remote files.
        If `True`, will use the token generated when running
        `transformers-cli login` (stored in `~/.huggingface`). Will
        default to `True` if `repo_url` is not specified.
    git_user (`str`, *optional*):
        will override the `git config user.name` for committing and
        pushing files to the hub.
    git_email (`str`, *optional*):
        will override the `git config user.email` for committing and
        pushing files to the hub.
    config (`dict`, *optional*):
        Configuration object to be saved alongside the model weights.


Returns:
    The url of the commit of your model in the given repository.

In [31]:
show_doc(CycleGAN.from_pretrained)

<h4 id="ModelHubMixin.from_pretrained" class="doc_header"><code>ModelHubMixin.from_pretrained</code><a href="huggingface_hub/hub_mixin.py#L73" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelHubMixin.from_pretrained</code>(**`pretrained_model_name_or_path`**:`str`, **`force_download`**:`bool`=*`False`*, **`resume_download`**:`bool`=*`False`*, **`proxies`**:`typing.Dict`=*`None`*, **`use_auth_token`**:`Optional`\[`str`\]=*`None`*, **`cache_dir`**:`Optional`\[`str`\]=*`None`*, **`local_files_only`**:`bool`=*`False`*, **\*\*`model_kwargs`**)

Instantiate a pretrained PyTorch model from a pre-trained model
        configuration from huggingface-hub. The model is set in
        evaluation mode by default using `model.eval()` (Dropout modules
        are deactivated). To train the model, you should first set it
        back in training mode with `model.train()`.

        Parameters:
            pretrained_model_name_or_path (`str` or `os.PathLike`):
                Can be either:
                    - A string, the `model id` of a pretrained model
                      hosted inside a model repo on huggingface.co.
                      Valid model ids can be located at the root-level,
                      like `bert-base-uncased`, or namespaced under a
                      user or organization name, like
                      `dbmdz/bert-base-german-cased`.
                    - You can add `revision` by appending `@` at the end
                      of model_id simply like this:
                      `dbmdz/bert-base-german-cased@main` Revision is
                      the specific model version to use. It can be a
                      branch name, a tag name, or a commit id, since we
                      use a git-based system for storing models and
                      other artifacts on huggingface.co, so `revision`
                      can be any identifier allowed by git.
                    - A path to a `directory` containing model weights
                      saved using
                      [`~transformers.PreTrainedModel.save_pretrained`],
                      e.g., `./my_model_directory/`.
                    - `None` if you are both providing the configuration
                      and state dictionary (resp. with keyword arguments
                      `config` and `state_dict`).
            force_download (`bool`, *optional*, defaults to `False`):
                Whether to force the (re-)download of the model weights
                and configuration files, overriding the cached versions
                if they exist.
            resume_download (`bool`, *optional*, defaults to `False`):
                Whether to delete incompletely received files. Will
                attempt to resume the download if such a file exists.
            proxies (`Dict[str, str]`, *optional*):
                A dictionary of proxy servers to use by protocol or
                endpoint, e.g., `{'http': 'foo.bar:3128',
                'http://hostname': 'foo.bar:4012'}`. The proxies are
                used on each request.
            use_auth_token (`str` or `bool`, *optional*):
                The token to use as HTTP bearer authorization for remote
                files. If `True`, will use the token generated when
                running `transformers-cli login` (stored in
                `~/.huggingface`).
            cache_dir (`Union[str, os.PathLike]`, *optional*):
                Path to a directory in which a downloaded pretrained
                model configuration should be cached if the standard
                cache should not be used.
            local_files_only(`bool`, *optional*, defaults to `False`):
                Whether to only look at local files (i.e., do not try to
                download the model).
            model_kwargs (`Dict`, *optional*):
                model_kwargs will be passed to the model during
                initialization

        <Tip>

        Passing `use_auth_token=True` is required when you want to use a
        private model.

        </Tip>

### Quick model tests

Again, let's check that the model can be called sucsessfully and outputs the correct shapes.


In [32]:
cyclegan_model = CycleGAN()
img1 = torch.randn(4,3,256,256)
img2 = torch.randn(4,3,256,256)

In [33]:
%%time
with torch.no_grad(): cyclegan_output = cyclegan_model((img1,img2))

CPU times: user 1min 15s, sys: 6.67 s, total: 1min 22s
Wall time: 2.25 s


In [34]:
test_eq(len(cyclegan_output),4)
for output_batch in cyclegan_output:
    test_eq(output_batch.shape,img1.shape)

In [35]:
cyclegan_model.push_to_hub('upit-cyclegan-test')

Cloning https://huggingface.co/tmabraham/upit-cyclegan-test into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/108M [00:00<?, ?B/s]

To https://huggingface.co/tmabraham/upit-cyclegan-test
   a41e9e0..2331f7d  main -> main



'https://huggingface.co/tmabraham/upit-cyclegan-test/commit/2331f7d345d719ac1fdfb10b2cddf58abd7931bb'

In [38]:
#hide_output
cyclegan_model.from_pretrained('tmabraham/upit-cyclegan-test')

config.json not found in HuggingFace Hub


CycleGAN(
  (D_A): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
  (D_B): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=

In [39]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_models.cyclegan.ipynb.
Converted 01b_models.junyanz.ipynb.
Converted 02_data.unpaired.ipynb.
Converted 03_train.cyclegan.ipynb.
Converted 04_inference.cyclegan.ipynb.
Converted 05_metrics.ipynb.
Converted 06_tracking.wandb.ipynb.
Converted 07_models.dualgan.ipynb.
Converted 08_train.dualgan.ipynb.
Converted 09_models.ganilla.ipynb.
Converted index.ipynb.
